In [2]:
using DataFrames, DataFramesMeta, CSV
using NPZ
using Random, Statistics
using DSP,FFTW #, AbstractFFTs,Interpolations
# using Wavelets, ContinuousWavelets
using ProgressMeter
using BenchmarkTools

using PyPlot
ENV["COLUMNS"] = 700
plt.style.use("seaborn-whitegrid");
rcParams = PyPlot.PyDict(PyPlot.matplotlib."rcParams")
rcParams["font.size"] = 12
rcParams["figure.figsize"] = [16,16]
rcParams["figure.dpi"] = 220;

In [113]:
BASE_DIR = "/home/yuri/kaggle/g2net-gravitational-wave-detection/"

"/home/yuri/kaggle/g2net-gravitational-wave-detection/"

In [114]:
train_label_df = CSV.read(joinpath(BASE_DIR, "training_labels.csv"), DataFrame)
train_label_df[:, :fname] = train_label_df[:, :id] .* ".npy"
const sampling_rate = 2048
const dt = 1 / sampling_rate

0.00048828125

In [115]:
train_label_df

,id,target,fname
,String,Int64,String
1,00000e74ad,1,00000e74ad.npy
2,00001f4945,0,00001f4945.npy
3,0000661522,0,0000661522.npy
4,00007a006a,0,00007a006a.npy
5,0000a38978,1,0000a38978.npy
6,0000bb9f3e,1,0000bb9f3e.npy
7,0000c3b9c9,0,0000c3b9c9.npy
8,0000d61b7b,1,0000d61b7b.npy
9,0001016d12,1,0001016d12.npy


In [116]:
const max_safe_exponent = 700.0

function fade!(signal, fade_length=0.06)
    n = length(signal)
    n0 = floor(Int, length(signal) * fade_length)
    n1 = n - n0
    t = collect(1:n)
    
    sigmoid(x)=1 ./(1 .+exp(-14 .*(x .-0.5)))
    for i in 1:n0
        signal[i] = signal[i] * sigmoid(i/n0)
    end
    for i in n1:n
        signal[i] = signal[i] * sigmoid((n-i)/(n-n1))
    end
    signal
end

function read_npy_data!(path, res, num)
    data = npzread(path) |> x-> reshape(x, 3, sampling_rate*2) |> transpose |> Array
    for i in 1:3
        data[:,i] = fade!(data[:,i])
        res[(num-1)*4096+1:num*4096,i] .= data[:,i]
    end
    # res = cat(res, data,dims=1)
end

test_data = zeros(4096,3)
@benchmark read_npy_data!(joinpath(BASE_DIR, "train/0/0/0/00000e74ad.npy"), test_data, 1)
# plt.plot(1:4096, test_data[:, 1])



BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  139.403 μs …  12.948 ms  ┊ GC (min … max):  0.00% … 93.68%
 Time  (median):     338.354 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   397.155 μs ± 783.402 μs  ┊ GC (mean ± σ):  13.62% ±  6.70%

  █                                                              
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▁▁  ▁ ▁▁▁▁▁▁ ▁
  688 μs           Histogram: frequency by time          287 μs <

 Memory estimate: 679.67 KiB, allocs estimate: 137.

In [35]:
function noise_detect(h, h̃, number_of_chunks, overlap_rate)
    points_per_chunk = 2^floor(Int, log2(length(h)/number_of_chunks))

    s = welch_pgram(h, points_per_chunk, floor(Int, points_per_chunk*overlap_rate), fs=sampling_rate, window=hanning)
    f_noise, noise_welch = s.freq, s.power

    noise_spectral_density = sqrt.(2 * length(h) .* noise_welch ./ sampling_rate)
    nodes = (f_noise,)
    itp = interpolate(nodes, noise_spectral_density, Gridded(Linear()))
    noise_spectral_density_interpolator = itp(frequencies);

    h̃_equalized = h̃ ./ noise_spectral_density_interpolator
    h_equalized = (sampling_rate * FFTW.irfft(h̃_equalized, length(t)));

    return h_equalized, h̃_equalized
end

noise_detect (generic function with 1 method)

In [119]:
function check_in_sig(fnames, df, target)
    res = String[]
    for fname in fnames
        if first(df[df[!, :fname] .== fname, :target]) == target
            push!(res, fname)
        end
    end
    res
end

@show(check_in_sig(["00000e74ad.npy", "00001f4945.npy"], train_label_df, 1))
@show(check_in_sig(["00000e74ad.npy", "00001f4945.npy"], train_label_df, 0))

check_in_sig(["00000e74ad.npy", "00001f4945.npy"], train_label_df, 1) = ["00000e74ad.npy"]
check_in_sig(["00000e74ad.npy", "00001f4945.npy"], train_label_df, 0) = ["00001f4945.npy"]


1-element Array{String,1}:
 "00001f4945.npy"

In [104]:
a = 1.2
arr = Float16[]
push!(arr,a)

1-element Array{Float16,1}:
 1.2

In [111]:
const number_of_chunks = 16
function main(train_test)
    out_path = joinpath(BASE_DIR, "dataset/train_equalized")
    @show(length(readdir(joinpath(BASE_DIR, "train"), join=true)))
    dir_0 = joinpath(BASE_DIR, train_test)
    for f1 in readdir(dir_0, join=false)
        dir_1 = joinpath(dir_0, f1)
        @showprogress for f2 in readdir(dir_1, join=false)
            dir_2 = joinpath(dir_1, f2)
            for f3 in readdir(dir_2, join=false)
                dir_3 = joinpath(dir_2, f3)
                for (target, fname_add) in zip([1, 0], ["in_sig.npy", "no_sig.npy"]) 
                    file_names = readdir(dir_3, join=false)
                    file_names = check_in_sig(file_names, train_label_df, target)
                    file_paths = String[]
                    # @show(target, file_names)
                    for file_name in file_names
                        push!(file_paths, joinpath(dir_3, file_name))
                    end
               
                    data = Array{Float64, 2}(undef, 4096*length(file_paths), 3)
                    for (num, file_path) in enumerate(file_paths)
                        read_npy_data!(file_path, data, num)
                    end
                    ##  main
                    h_equalized_filtered_arr = Float32[]
                    h̃_equalized_filtered_arr = ComplexF32[]
                    for i in 1:3 ## LIGO LIGO Virgo
                        h = data[:, i]
                        global t = collect(1:length(h)) / sampling_rate
                        global frequencies = FFTW.rfftfreq(length(h), sampling_rate)
                        h̃ = dt * FFTW.rfft(h) 
                        h_equalized, h̃_equalized= noise_detect(h, h̃, number_of_chunks, 0.5)
                        responsetype = Bandpass(35, 300, fs=sampling_rate)
                        h_equalized_filtered = filt(digitalfilter(responsetype, Butterworth(4)), h_equalized)
                        h̃_equalized_filtered = dt * FFTW.rfft(h_equalized_filtered) 
                        # @show(size(h))
                        # @show(size(h̃))
                        # @show(size(h_equalized_filtered))
                        # @show(size(h̃_equalized_filtered))
                        append!(h_equalized_filtered_arr, Float32.(h_equalized_filtered))
                        append!(h̃_equalized_filtered_arr, ComplexF32.(h̃_equalized_filtered))
                    end
                    h_equalized_filtered_arr = reshape(h_equalized_filtered_arr, (Int(length(h_equalized_filtered_arr)/3), 3))
                    h̃_equalized_filtered_arr = reshape(h̃_equalized_filtered_arr, (Int(length(h̃_equalized_filtered_arr)/3), 3))
                    # save_output
                    # split each npy file and save
                    # @show(size(h_equalized_filtered_arr))
                    npzwrite(joinpath(out_path, "$(f1)_$(f2)_$(f3)_$(fname_add)"), 
                        h_equalized_filtered_arr |>  transpose |> Array)
                    end # target, fname_add
                end # f3
        end
        return 
    end
end
 
main("train")

length(readdir(joinpath(BASE_DIR, "train"), join = true)) = 16


Progress: 100%|█████████████████████████████████████████| Time: 0:06:45


In [5]:
a = ["abc", "cdf.npy"]
"cdf.npy" in a

true

In [62]:
4096 * 16

65536

In [ ]:
const number_of_chunks = 16
function main(train_test)
    out_path = joinpath(BASE_DIR, "dataset/train_equalized")
    @show(length(readdir(joinpath(BASE_DIR, "train"), join=true)))
    dir_0 = joinpath(BASE_DIR, train_test)
    for f1 in readdir(dir_0, join=false)
        dir_1 = joinpath(dir_0, f1)
        @showprogress for f2 in readdir(dir_1, join=false)
            dir_2 = joinpath(dir_1, f2)
            for f3 in readdir(dir_2, join=false)
                dir_3 = joinpath(dir_2, f3)
                for (target, fname_add) in zip([1, 0], ["in_sig.npy", "no_sig.npy"]) 
                    file_names = readdir(dir_3, join=false)
                    file_names = check_in_sig(file_names, train_label_df, target)
                    file_paths = String[]
                    # @show(target, file_names)
                    for file_name in file_names
                        push!(file_paths, joinpath(dir_3, file_name))
                    end
               
                    data = Array{Float64, 2}(undef, 4096*length(file_paths), 3)
                    for (num, file_path) in enumerate(file_paths)
                        read_npy_data!(file_path, data, num)
                    end
                    ##  main
                    h_equalized_filtered_arr = Float32[]
                    h̃_equalized_filtered_arr = ComplexF32[]
                    for i in 1:3 ## LIGO LIGO Virgo
                        h = data[:, i]
                        global t = collect(1:length(h)) / sampling_rate
                        global frequencies = FFTW.rfftfreq(length(h), sampling_rate)
                        h̃ = dt * FFTW.rfft(h) 
                        h_equalized, h̃_equalized= noise_detect(h, h̃, number_of_chunks, 0.5)
                        responsetype = Bandpass(35, 300, fs=sampling_rate)
                        h_equalized_filtered = filt(digitalfilter(responsetype, Butterworth(4)), h_equalized)
                        h̃_equalized_filtered = dt * FFTW.rfft(h_equalized_filtered) 
                        append!(h_equalized_filtered_arr, Float32.(h_equalized_filtered))
                        append!(h̃_equalized_filtered_arr, ComplexF32.(h̃_equalized_filtered))
                    end
                    h_equalized_filtered_arr = reshape(h_equalized_filtered_arr, (Int(length(h_equalized_filtered_arr)/3), 3))
                    h̃_equalized_filtered_arr = reshape(h̃_equalized_filtered_arr, (Int(length(h̃_equalized_filtered_arr)/3), 3))
                    npzwrite(joinpath(out_path, "$(f1)_$(f2)_$(f3)_$(fname_add)"), 
                        h_equalized_filtered_arr |>  transpose |> Array)
                    end # target, fname_add
                end # f3
        end
    end
end
 
main("train")

In [124]:

println(isfile("LOSC_Event_tutorial.ipynb"))


true


In [137]:
exp.(-0.25*(0:13))

14-element Array{Float64,1}:
 1.0
 0.7788007830714049
 0.6065306597126334
 0.4723665527410147
 0.36787944117144233
 0.2865047968601901
 0.22313016014842982
 0.1737739434504451
 0.1353352832366127
 0.10539922456186433
 0.0820849986238988
 0.06392786120670757
 0.049787068367863944
 0.03877420783172201